In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Reshape
from tensorflow.keras.models import Model
import numpy as np
import networkx as nx

In [2]:
def first_s2v_iter(adjmat, prev_embeddings, theta2):
    sum_neighbor_rows = tf.einsum('aij,jk->aik', adjmat, prev_embeddings)
    return tf.nn.relu(sum_neighbor_rows * theta2)

def other_s2v_iter(adjmat, prev_embeddings, theta2):
    sum_neighbor_rows = tf.einsum('aij,ajk->aik', adjmat, prev_embeddings)
    return tf.nn.relu(sum_neighbor_rows * theta2)

def s2v_four_times(adjmat, initial_embeddings, theta2):
    curr_embed = first_s2v_iter(adjmat, initial_embeddings, theta2)
    for i in range(3):
        curr_embed = other_s2v_iter(adjmat, curr_embed, theta2)
    return curr_embed

In [3]:
class S2VLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, **kwargs):
        self.embedding_dim = embedding_dim
        super(S2VLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.theta2 = self.add_weight(name='theta2', shape=tf.TensorShape([1,self.embedding_dim]), initializer='uniform', trainable=True)
        self.initial_embeddings = self.add_weight(name='init_theta', shape=tf.TensorShape([input_shape[1],self.embedding_dim]), initializer='ones', trainable=False)
        super(S2VLayer, self).build(input_shape)
        
    def call(self, adjmat):
        return s2v_four_times(adjmat, self.initial_embeddings, self.theta2)
    
    def compute_output_shape(self, input_shape):
        return tf.TensorShape([input_shape[0], input_shape[1], self.embedding_dim])

In [4]:
class S2VGraph(object):
    def __init__(self, g, node_tags, label):
        self.num_nodes = len(node_tags)
        self.node_tags = node_tags
        self.label = label
        self.g = g
        x, y = zip(*g.edges())
        self.num_edges = len(x)        
        self.edge_pairs = np.ndarray(shape=(self.num_edges, 2), dtype=np.int32)
        self.edge_pairs[:, 0] = x
        self.edge_pairs[:, 1] = y
        self.edge_pairs = self.edge_pairs.flatten()

In [5]:
def load_data():
    print('loading data')

    g_list = []
    label_dict = {}
    feat_dict = {}
    fold = 1
    with open('./data/%s/%s.txt' % ('NCI1', 'NCI1'), 'r') as f:
        n_g = int(f.readline().strip())
        for i in range(n_g):
            row = f.readline().strip().split()
            n, l = [int(w) for w in row]
            if not l in label_dict:
                mapped = len(label_dict)
                label_dict[l] = mapped
            g = nx.Graph()
            node_tags = []
            n_edges = 0
            for j in range(n):
                g.add_node(j)
                row = f.readline().strip().split()
                row = [int(w) for w in row]
                if not row[0] in feat_dict:
                    mapped = len(feat_dict)
                    feat_dict[row[0]] = mapped
                node_tags.append(feat_dict[row[0]])

                n_edges += row[1]
                for k in range(2, len(row)):
                    g.add_edge(j, row[k])
            assert len(g.edges()) * 2 == n_edges
            assert len(g) == n
            g_list.append(S2VGraph(g, node_tags, l))
    for g in g_list:
        g.label = label_dict[g.label]
    print('# classes: %d' % len(label_dict))
    print('# node features: %d' % len(feat_dict))

    train_idxes = np.loadtxt('./data/%s/10fold_idx/train_idx-%d.txt' % ('NCI1', fold), dtype=np.int32).tolist()
    test_idxes = np.loadtxt('./data/%s/10fold_idx/test_idx-%d.txt' % ('NCI1', fold), dtype=np.int32).tolist()

    return [g_list[i] for i in train_idxes], [g_list[i] for i in test_idxes]
    

In [6]:
graph_train, graph_test = load_data()

loading data
# classes: 2
# node features: 37


In [7]:
graph_train_shortened = [g for g in graph_train if g.num_nodes <= 50]
graph_test_shortened = [g for g in graph_test if g.num_nodes <= 50]

In [8]:
def adjmat(gr):
    return nx.adjacency_matrix(gr).toarray().astype('float32')
def zero_padded_adjmat(graph, size):
    unpadded = adjmat(graph)
    padded = np.zeros((size, size))
    padded[0:unpadded.shape[0], 0:unpadded.shape[1]] = unpadded
    padded = np.reshape(padded, (padded.shape[0], padded.shape[1], 1))
    return padded

In [9]:
graph_train_adjmat = np.stack([zero_padded_adjmat(g.g, 50) for g in graph_train_shortened])
graph_train_labels = np.expand_dims(np.stack([g.label for g in graph_train_shortened]).astype('float32'),axis=1)

graph_test_adjmat = np.stack([zero_padded_adjmat(g.g, 50) for g in graph_test_shortened])
graph_test_labels = np.expand_dims(np.stack([g.label for g in graph_test_shortened]).astype('float32'),axis=1)

In [10]:
tf.enable_eager_execution()

In [14]:
def s2v_model(input_size=50):
    input_im = Input(shape=(input_size, input_size,1))
    squeezed = Reshape((input_size, input_size))(input_im)
    l1 = S2VLayer(32)(squeezed)
    out = Flatten()(l1)
    out = Dense(128, activation='relu')(out)
    out = Dense(1, activation='sigmoid')(out)
    s2v_model = Model(input_im, out)
    return s2v_model

In [15]:
s2v = s2v_model()
s2v.compile(optimizer=tf.train.AdadeltaOptimizer(), loss='binary_crossentropy')

In [13]:
s2v.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 1)         0         
_________________________________________________________________
reshape (Reshape)            (None, 50, 50)            0         
_________________________________________________________________
s2v_layer (S2VLayer)         (None, 50, 32)            1632      
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 104,161
Trainable params: 102,561
Non-trainable params: 1,600
_________________________________________________________________


In [16]:
s2v.fit(graph_train_adjmat, graph_train_labels, epochs=5000, batch_size=1000, shuffle=True, validation_data=(graph_test_adjmat, graph_test_labels))

Train on 3422 samples, validate on 380 samples
Epoch 1/5000
3422/3422 [==============================] - 0s 50us/step - loss: 4.3204 - val_loss: 3.4685
Epoch 2/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.3009 - val_loss: 3.4518
Epoch 3/5000
3422/3422 [==============================] - 0s 17us/step - loss: 4.2809 - val_loss: 3.4336
Epoch 4/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.2608 - val_loss: 3.4163
Epoch 5/5000
3422/3422 [==============================] - 0s 17us/step - loss: 4.2407 - val_loss: 3.3984
Epoch 6/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.2204 - val_loss: 3.3796
Epoch 7/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.1995 - val_loss: 3.3611
Epoch 8/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.1785 - val_loss: 3.3432
Epoch 9/5000
3422/3422 [==============================] - 0s 18us/step - loss: 4.1579 - val_loss: 3.3261
Epoch 10

Epoch 78/5000
3422/3422 [==============================] - 0s 17us/step - loss: 2.7076 - val_loss: 2.1153
Epoch 79/5000
3422/3422 [==============================] - 0s 17us/step - loss: 2.6893 - val_loss: 2.1014
Epoch 80/5000
3422/3422 [==============================] - 0s 17us/step - loss: 2.6706 - val_loss: 2.0883
Epoch 81/5000
3422/3422 [==============================] - 0s 18us/step - loss: 2.6524 - val_loss: 2.0749
Epoch 82/5000
3422/3422 [==============================] - 0s 19us/step - loss: 2.6343 - val_loss: 2.0620
Epoch 83/5000
3422/3422 [==============================] - 0s 18us/step - loss: 2.6164 - val_loss: 2.0490
Epoch 84/5000
3422/3422 [==============================] - 0s 18us/step - loss: 2.5988 - val_loss: 2.0364
Epoch 85/5000
3422/3422 [==============================] - 0s 20us/step - loss: 2.5810 - val_loss: 2.0237
Epoch 86/5000
3422/3422 [==============================] - 0s 18us/step - loss: 2.5635 - val_loss: 2.0111
Epoch 87/5000
3422/3422 [=====================

Epoch 155/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.6991 - val_loss: 1.4231
Epoch 156/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.6910 - val_loss: 1.4176
Epoch 157/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.6828 - val_loss: 1.4123
Epoch 158/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.6749 - val_loss: 1.4069
Epoch 159/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.6670 - val_loss: 1.4017
Epoch 160/5000
3422/3422 [==============================] - 0s 19us/step - loss: 1.6592 - val_loss: 1.3964
Epoch 161/5000
3422/3422 [==============================] - 0s 19us/step - loss: 1.6516 - val_loss: 1.3913
Epoch 162/5000
3422/3422 [==============================] - 0s 17us/step - loss: 1.6442 - val_loss: 1.3861
Epoch 163/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.6367 - val_loss: 1.3810
Epoch 164/5000
3422/3422 [===========

3422/3422 [==============================] - 0s 18us/step - loss: 1.3071 - val_loss: 1.1542
Epoch 232/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.3041 - val_loss: 1.1521
Epoch 233/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.3012 - val_loss: 1.1502
Epoch 234/5000
3422/3422 [==============================] - 0s 17us/step - loss: 1.2984 - val_loss: 1.1482
Epoch 235/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.2956 - val_loss: 1.1461
Epoch 236/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.2928 - val_loss: 1.1444
Epoch 237/5000
3422/3422 [==============================] - 0s 17us/step - loss: 1.2902 - val_loss: 1.1423
Epoch 238/5000
3422/3422 [==============================] - 0s 19us/step - loss: 1.2874 - val_loss: 1.1402
Epoch 239/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.2847 - val_loss: 1.1381
Epoch 240/5000
3422/3422 [==========================

3422/3422 [==============================] - 0s 20us/step - loss: 1.1589 - val_loss: 1.0570
Epoch 308/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.1577 - val_loss: 1.0564
Epoch 309/5000
3422/3422 [==============================] - 0s 21us/step - loss: 1.1563 - val_loss: 1.0554
Epoch 310/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.1550 - val_loss: 1.0544
Epoch 311/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.1536 - val_loss: 1.0533
Epoch 312/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.1524 - val_loss: 1.0520
Epoch 313/5000
3422/3422 [==============================] - 0s 18us/step - loss: 1.1511 - val_loss: 1.0517
Epoch 314/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.1498 - val_loss: 1.0512
Epoch 315/5000
3422/3422 [==============================] - 0s 20us/step - loss: 1.1485 - val_loss: 1.0501
Epoch 316/5000
3422/3422 [==========================

...

Epoch 4639/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6437
Epoch 4640/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6436
Epoch 4641/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6433
Epoch 4642/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6430
Epoch 4643/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5763 - val_loss: 0.6425
Epoch 4644/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6429
Epoch 4645/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5763 - val_loss: 0.6430
Epoch 4646/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5762 - val_loss: 0.6429
Epoch 4647/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5762 - val_loss: 0.6427
Epoch 4648/5000
3422/3422 [=

Epoch 4715/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5754 - val_loss: 0.6421
Epoch 4716/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5754 - val_loss: 0.6427
Epoch 4717/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5754 - val_loss: 0.6429
Epoch 4718/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5754 - val_loss: 0.6429
Epoch 4719/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5753 - val_loss: 0.6430
Epoch 4720/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5753 - val_loss: 0.6428
Epoch 4721/5000
3422/3422 [==============================] - 0s 18us/step - loss: 0.5753 - val_loss: 0.6430
Epoch 4722/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5753 - val_loss: 0.6430
Epoch 4723/5000
3422/3422 [==============================] - 0s 17us/step - loss: 0.5753 - val_loss: 0.6425
Epoch 4724/5000
3422/3422 [=

Epoch 4791/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5745 - val_loss: 0.6425
Epoch 4792/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5746 - val_loss: 0.6427
Epoch 4793/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5745 - val_loss: 0.6424
Epoch 4794/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5745 - val_loss: 0.6422
Epoch 4795/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5744 - val_loss: 0.6421
Epoch 4796/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5745 - val_loss: 0.6419
Epoch 4797/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5744 - val_loss: 0.6420
Epoch 4798/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5744 - val_loss: 0.6416
Epoch 4799/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5744 - val_loss: 0.6414
Epoch 4800/5000
3422/3422 [=

Epoch 4867/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5736 - val_loss: 0.6420
Epoch 4868/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5736 - val_loss: 0.6420
Epoch 4869/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5736 - val_loss: 0.6416
Epoch 4870/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5735 - val_loss: 0.6418
Epoch 4871/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5736 - val_loss: 0.6417
Epoch 4872/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5736 - val_loss: 0.6411
Epoch 4873/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5735 - val_loss: 0.6410
Epoch 4874/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5735 - val_loss: 0.6410
Epoch 4875/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5735 - val_loss: 0.6407
Epoch 4876/5000
3422/3422 [=

Epoch 4943/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5727 - val_loss: 0.6412
Epoch 4944/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5727 - val_loss: 0.6408
Epoch 4945/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5727 - val_loss: 0.6409
Epoch 4946/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5727 - val_loss: 0.6413
Epoch 4947/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5727 - val_loss: 0.6413
Epoch 4948/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5727 - val_loss: 0.6411
Epoch 4949/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5727 - val_loss: 0.6414
Epoch 4950/5000
3422/3422 [==============================] - 0s 21us/step - loss: 0.5727 - val_loss: 0.6409
Epoch 4951/5000
3422/3422 [==============================] - 0s 20us/step - loss: 0.5726 - val_loss: 0.6408
Epoch 4952/5000
3422/3422 [=

In [18]:
s2v.predict(graph_test_adjmat[0:1,:,:,:])

array([[0.23739757]], dtype=float32)

In [20]:
graph_test_labels[0]

array([1.], dtype=float32)